In [ ]:
## Starter Code

#### Note: We also provide feature information on the data as a .csv file for easy lookup throughout the notebook:

import pandas as pd

data_info = pd.read_csv('../DATA/lending_club_info.csv',index_col='LoanStatNew')

print(data_info.loc['revol_util']['Description'])

def feat_info(col_name):
    print(data_info.loc[col_name]['Description'])

feat_info('mort_acc')

## Loading the data and other imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# might be needed depending on your version of Jupyter
%matplotlib inline

df = pd.read_csv('../DATA/lending_club_loan_two.csv')

df.info()

# Project Tasks

**Complete the tasks below! Keep in mind is usually more than one way to complete the task! Enjoy**

-----
------

# Section 1: Exploratory Data Analysis

**OVERALL GOAL: Get an understanding for which variables are important, view summary statistics, and visualize the data**


----

**TASK: Since we will be attempting to predict loan_status, create a countplot as shown below.**

# CODE HERE

sns.countplot(df['loan_status'])

**TASK: Create a histogram of the loan_amnt column.**

# CODE HERE

plt.figure(figsize=(10,6))
sns.distplot(df['loan_amnt'], kde=False,bins = 30)
plt.xlim(0,45000)

**TASK: Let's explore correlation between the continuous feature variables. Calculate the correlation between all continuous numeric variables using .corr() method.**

# CODE HERE

df.corr()

**TASK: Visualize this using a heatmap. Depending on your version of matplotlib, you may need to manually adjust the heatmap.**

* [Heatmap info](https://seaborn.pydata.org/generated/seaborn.heatmap.html#seaborn.heatmap)
* [Help with resizing](https://stackoverflow.com/questions/56942670/matplotlib-seaborn-first-and-last-row-cut-in-half-of-heatmap-plot)

# CODE HERE

cor = df.corr()
plt.figure(figsize=(12,10))
sns.heatmap(cor,annot=True)

**TASK: You should have noticed almost perfect correlation with the "installment" feature. Explore this feature further. Print out their descriptions and perform a scatterplot between them. Does this relationship make sense to you? Do you think there is duplicate information here?**

# CODE HERE





sns.scatterplot(x='installment',y='loan_amnt', data=df)

**TASK: Create a boxplot showing the relationship between the loan_status and the Loan Amount.**

# CODE HERE

sns.boxplot(x='loan_status',y='loan_amnt', data=df)

**TASK: Calculate the summary statistics for the loan amount, grouped by the loan_status.**

# CODE HERE

df.groupby(df['loan_status'])['loan_amnt'].describe()

**TASK: Let's explore the Grade and SubGrade columns that LendingClub attributes to the loans. What are the unique possible grades and subgrades?**

# CODE HERE

sorted(df['grade'].unique())

sorted(df['sub_grade'].unique())

**TASK: Create a countplot per grade. Set the hue to the loan_status label.**

# CODE HERE

sns.countplot(df['grade'],hue=df['loan_status'])

**TASK: Display a count plot per subgrade. You may need to resize for this plot and [reorder](https://seaborn.pydata.org/generated/seaborn.countplot.html#seaborn.countplot) the x axis. Feel free to edit the color palette. Explore both all loans made per subgrade as well being separated based on the loan_status. After creating this plot, go ahead and create a similar plot, but set hue="loan_status"**

#CODE HERE

plt.figure(figsize=(15,6))
sns.countplot(df['sub_grade'],order=sorted(df['sub_grade'].unique()))

# CODE HERE

plt.figure(figsize=(15,6))
sns.countplot(df['sub_grade'],order=sorted(df['sub_grade'].unique()))

**TASK: It looks like F and G subgrades don't get paid back that often. Isloate those and recreate the countplot just for those subgrades.**

# CODE HERE

df_FG = df[(df['sub_grade'].str.contains('F')) | (df['sub_grade'].str.contains('G'))]
sns.countplot(df_FG['sub_grade'], order=sorted(df_FG['sub_grade'].unique()), hue=df_FG['loan_status'])

**TASK: Create a new column called 'loan_repaid' which will contain a 1 if the loan status was "Fully Paid" and a 0 if it was "Charged Off".**

# CODE HERE

def repaid(x):
    if x == 'Fully Paid':
        return 1
    else:
        return 0

df['loan_repaid'] = df['loan_status'].apply(repaid)

df[['loan_repaid','loan_status']]

**CHALLENGE TASK: (Note this is hard, but can be done in one line!) Create a bar plot showing the correlation of the numeric features to the new loan_repaid column. [Helpful Link](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.bar.html)**

#CODE HERE

df.corr()['loan_repaid'][:-1].sort_values().plot(kind='bar')

---
---
# Section 2: Data PreProcessing

**Section Goals: Remove or fill any missing data. Remove unnecessary or repetitive features. Convert categorical string features to dummy variables.**





# Missing Data

**Let's explore this missing data columns. We use a variety of factors to decide whether or not they would be useful, to see if we should keep, discard, or fill in the missing data.**

**TASK: What is the length of the dataframe?**

# CODE HERE

len(df)

**TASK: Create a Series that displays the total count of missing values per column.**

# CODE HERE

df.isnull().sum()

**TASK: Convert this Series to be in term of percentage of the total DataFrame**

# CODE HERE

(df.isnull().sum())/len(df)*100

**TASK: Let's examine emp_title and emp_length to see whether it will be okay to drop them. Print out their feature information using the feat_info() function from the top of this notebook.**

# CODE HERE



**TASK: How many unique employment job titles are there?**

# CODE HERE

len(df['emp_title'].unique())

df['emp_title'].value_counts()

**TASK: Realistically there are too many unique job titles to try to convert this to a dummy variable feature. Let's remove that emp_title column.**

# CODE HERE

df = df.drop('emp_title',axis=1)

**TASK: Create a count plot of the emp_length feature column. Challenge: Sort the order of the values.**

# CODE HERE

sorted(df['emp_length'].dropna().unique())

emp_length_order = [ '< 1 year',
                      '1 year',
                     '2 years',
                     '3 years',
                     '4 years',
                     '5 years',
                     '6 years',
                     '7 years',
                     '8 years',
                     '9 years',
                     '10+ years']

plt.figure(figsize=(10,6))
sns.countplot(df['emp_length'],order=emp_length_order)

**TASK: Plot out the countplot with a hue separating Fully Paid vs Charged Off**

# CODE HERE

plt.figure(figsize=(10,6))
sns.countplot(df['emp_length'],order=emp_length_order,hue=df['loan_status'])

**CHALLENGE TASK: This still doesn't really inform us if there is a strong relationship between employment length and being charged off, what we want is the percentage of charge offs per category. Essentially informing us what percent of people per employment category didn't pay back their loan. There are a multitude of ways to create this Series. Once you've created it, see if visualize it with a [bar plot](https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.DataFrame.plot.html). This may be tricky, refer to solutions if you get stuck on creating this Series.**

# CODE HERE

co = df[df['loan_status']=='Charged Off'].groupby(['emp_length']).count()['loan_status']

fp = df[df['loan_status']=='Fully Paid'].groupby(['emp_length']).count()['loan_status']

co/fp



(co/fp).plot(kind='bar')

**TASK: Charge off rates are extremely similar across all employment lengths. Go ahead and drop the emp_length column.**

# CODE HERE

df = df.drop('emp_length',axis=1)

**TASK: Revisit the DataFrame to see what feature columns still have missing data.**





**TASK: Review the title column vs the purpose column. Is this repeated information?**

# CODE HERE



df['title'].head(10)

**TASK: The title column is simply a string subcategory/description of the purpose column. Go ahead and drop the title column.**

# CODE HERE

df = df.drop('title',axis=1)

---
**NOTE: This is one of the hardest parts of the project! Refer to the solutions video if you need guidance, feel free to fill or drop the missing values of the mort_acc however you see fit! Here we're going with a very specific approach.**


---
**TASK: Find out what the mort_acc feature represents**

# CODE HERE



**TASK: Create a value_counts of the mort_acc column.**

# CODE HERE

df['mort_acc'].value_counts()

**TASK: There are many ways we could deal with this missing data. We could attempt to build a simple model to fill it in, such as a linear model, we could just fill it in based on the mean of the other columns, or you could even bin the columns into categories and then set NaN as its own category. There is no 100% correct approach! Let's review the other columsn to see which most highly correlates to mort_acc**



df.corr()['mort_acc'].sort_values()

**TASK: Looks like the total_acc feature correlates with the mort_acc , this makes sense! Let's try this fillna() approach. We will group the dataframe by the total_acc and calculate the mean value for the mort_acc per total_acc entry. To get the result below:**



df.groupby('total_acc').mean()['mort_acc']

**CHALLENGE TASK: Let's fill in the missing mort_acc values based on their total_acc value. If the mort_acc is missing, then we will fill in that missing value with the mean value corresponding to its total_acc value from the Series we created above. This involves using an .apply() method with two columns. Check out the link below for more info, or review the solutions video/notebook.**

[Helpful Link](https://stackoverflow.com/questions/13331698/how-to-apply-a-function-to-two-columns-of-pandas-dataframe) 

# CODE HERE

acc_avg = df.groupby('total_acc').mean()['mort_acc']

acc_avg[2]

def check_nan(total_acc, mort_acc):
    if np.isnan(mort_acc):
        return acc_avg[total_acc]
    else:
        return mort_acc

df['mort_acc'] = df.apply(lambda x : check_nan(x['total_acc'],x['mort_acc']), axis=1)

df.isnull().sum()

**TASK: revol_util and the pub_rec_bankruptcies have missing data points, but they account for less than 0.5% of the total data. Go ahead and remove the rows that are missing those values in those columns with dropna().**

# CODE HERE

df.dropna(inplace=True)

df.isnull().sum()

## Categorical Variables and Dummy Variables

**We're done working with the missing data! Now we just need to deal with the string values due to the categorical columns.**

**TASK: List all the columns that are currently non-numeric. [Helpful Link](https://stackoverflow.com/questions/22470690/get-list-of-pandas-dataframe-columns-based-on-data-type)**

[Another very useful method call](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.select_dtypes.html)

# CODE HERE

df.select_dtypes(include='object').columns

---
**Let's now go through all the string features to see what we should do with them.**

---


### term feature

**TASK: Convert the term feature into either a 36 or 60 integer numeric data type using .apply() or .map().**

# CODE HERE

df['term'].value_counts()

def convert(x):
    if(x[1] == '3'):
        return 36
    else:
        return 60
df['term'] = df.apply(lambda x : convert(x['term']), axis=1)

### grade feature

**TASK: We already know grade is part of sub_grade, so just drop the grade feature.**

# CODE HERE

df = df.drop('grade',axis=1)

**TASK: Convert the subgrade into dummy variables. Then concatenate these new columns to the original dataframe. Remember to drop the original subgrade column and to add drop_first=True to your get_dummies call.**

# CODE HERE

subgrade_dummies = pd.get_dummies(df['sub_grade'],drop_first=True)

df = pd.concat([df.drop('sub_grade',axis=1),subgrade_dummies],axis=1)

df.columns

df.select_dtypes(include='object').columns

### verification_status, application_type,initial_list_status,purpose 
**TASK: Convert these columns: ['verification_status', 'application_type','initial_list_status','purpose'] into dummy variables and concatenate them with the original dataframe. Remember to set drop_first=True and to drop the original columns.**

# CODE HERE

dummies = pd.get_dummies(df[['verification_status', 'application_type','initial_list_status','purpose' ]],drop_first=True)
df = df.drop(['verification_status', 'application_type','initial_list_status','purpose'],axis=1)
df = pd.concat([df,dummies],axis=1)



### home_ownership
**TASK:Review the value_counts for the home_ownership column.**

#CODE HERE

df['home_ownership'].value_counts()

**TASK: Convert these to dummy variables, but [replace](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html) NONE and ANY with OTHER, so that we end up with just 4 categories, MORTGAGE, RENT, OWN, OTHER. Then concatenate them with the original dataframe. Remember to set drop_first=True and to drop the original columns.**

#CODE HERE

df['home_ownership']=df['home_ownership'].replace(['NONE', 'ANY'], 'OTHER')

dummies = pd.get_dummies(df['home_ownership'],drop_first=True)
df = df.drop('home_ownership',axis=1)
df = pd.concat([df,dummies],axis=1)

### address
**TASK: Let's feature engineer a zip code column from the address in the data set. Create a column called 'zip_code' that extracts the zip code from the address column.**

#CODE HERE

df['zip_code'] = df['address'].apply(lambda address:address[-5:])

**TASK: Now make this zip_code column into dummy variables using pandas. Concatenate the result and drop the original zip_code column along with dropping the address column.**

dummies = pd.get_dummies(df['zip_code'],drop_first=True)
df = df.drop(['zip_code','address'],axis=1)
df = pd.concat([df,dummies],axis=1)



### issue_d 

**TASK: This would be data leakage, we wouldn't know beforehand whether or not a loan would be issued when using our model, so in theory we wouldn't have an issue_date, drop this feature.**

#CODE HERE

df = df.drop('issue_d',axis=1)

### earliest_cr_line
**TASK: This appears to be a historical time stamp feature. Extract the year from this feature using a .apply function, then convert it to a numeric feature. Set this new data to a feature column called 'earliest_cr_year'.Then drop the earliest_cr_line feature.**

#CODE HERE

df['earliest_cr_year'] = df['earliest_cr_line'].apply(lambda date:int(date[-4:]))
df = df.drop('earliest_cr_line',axis=1)



## Train Test Split

**TASK: Import train_test_split from sklearn.**

from sklearn.model_selection import train_test_split

**TASK: drop the load_status column we created earlier, since its a duplicate of the loan_repaid column. We'll use the loan_repaid column since its already in 0s and 1s.**

# CODE HERE

df = df.drop('loan_status',axis=1)

**TASK: Set X and y variables to the .values of the features and label.**

#CODE HERE

X = df.drop('loan_repaid',axis=1).values
y = df['loan_repaid'].values

----
----

# OPTIONAL

## Grabbing a Sample for Training Time

### OPTIONAL: Use .sample() to grab a sample of the 490k+ entries to save time on training. Highly recommended for lower RAM computers or if you are not using GPU.

----
----

# df = df.sample(frac=0.1,random_state=101)
print(len(df))

**TASK: Perform a train/test split with test_size=0.2 and a random_state of 101.**

#CODE HERE

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=101)

## Normalizing the Data

**TASK: Use a MinMaxScaler to normalize the feature data X_train and X_test. Recall we don't want data leakge from the test set so we only fit on the X_train data.**

# CODE HERE

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

# Creating the Model

**TASK: Run the cell below to import the necessary Keras functions.**

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout

**TASK: Build a sequential model to will be trained on the data. You have unlimited options here, but here is what the solution uses: a model that goes 78 --> 39 --> 19--> 1 output neuron. OPTIONAL: Explore adding [Dropout layers](https://keras.io/layers/core/) [1](https://en.wikipedia.org/wiki/Dropout_(neural_networks)) [2](https://towardsdatascience.com/machine-learning-part-20-dropout-keras-layers-explained-8c9f6dc4c9ab)**





**TASK: Fit the model to the training data for at least 25 epochs. Also add in the validation data for later plotting. Optional: add in a batch_size of 256.**

# CODE HERE

model = Sequential()

model.add(Dense(79,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(30,activation='relu'))
model.add(Dropout(0.7))
model.add(Dense(15,activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam')

**TASK: OPTIONAL: Save your model.**

# CODE HERE





# Section 3: Evaluating Model Performance.

**TASK: Plot out the validation loss versus the training loss.**

# CODE HERE

model.fit(X_train,y_train,epochs=25,validation_data=(X_test,y_test),verbose=0)

model_loss = pd.DataFrame(model.history.history)
model_loss.plot()

**TASK: Create predictions from the X_test set and display a classification report and confusion matrix for the X_test set.**

# CODE HERE

predictions = model.predict_classes(X_test)

from sklearn.metrics import classification_report,confusion_matrix

print(classification_report(y_test,predictions))

print(confusion_matrix(y_test,predictions))

**TASK: Given the customer below, would you offer this person a loan?**

import random
random.seed(101)
random_ind = random.randint(0,len(df))

new_customer = pd.DataFrame(df.drop('loan_repaid',axis=1).iloc[random_ind])
new_customer

# CODE HERE

new_customer = scaler.transform(new_customer.transpose())
predict = model.predict_classes(new_customer)

**TASK: Now check, did this person actually end up paying back their loan?**

# CODE HERE

predict
df['loan_repaid'].iloc[random_ind]